In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from math import ceil
from CIoTS import *

In [3]:
max_ps = list(range(2, 22, 2))
runs = 20
dimensions = 4
data_length = 1000
alpha = 0.05

In [4]:
ics = ["aic", "bic", "hqic"]

In [14]:
p_estimations = pd.DataFrame(columns=['p' ,'method', 'mean_p', 'std_p'])
f1_scores = pd.DataFrame(columns=['p' ,'method', 'mean_f1', 'std_f1'])

In [15]:
for p in max_ps:
    incoming_edges = max(ceil(dimensions*p/3), 1)
    f1 = {ic: [] for ic in ics}
    f1['real'] = []
    
    p_est = {ic: [] for ic in ics}
    
    for i in range(runs):
        generator = CausalTSGenerator(dimensions=dimensions, max_p=p, data_length=data_length, incoming_edges=incoming_edges)
        ts = generator.generate()
        predicted_graph = pc_chen_modified(partial_corr_test, ts, p, alpha)
        
        f1['real'].append(evaluate_edges(generator.graph, predicted_graph)['f1-score'])
        
        var_ranking, var_scores = var_order_select(ts, 2*p, ics) 
        for ic in ics:
            predicted_graph = pc_chen(partial_corr_test, ts, var_ranking[ic][0], alpha)
        
            p_est[ic].append(var_ranking[ic][0])
            f1[ic].append(evaluate_edges(generator.graph, predicted_graph)['f1-score'])
    
    
    f1_scores = f1_scores.append({'p': p, 'method': 'real', 'mean_f1': np.mean(f1['real']), 'std_f1': np.std(f1['real'])},
                                 ignore_index=True)
    for ic in ics:
        p_estimations = p_estimations.append({'p': p, 'method': ic, 'mean_p': np.mean(p_est[ic]), 'std_p': np.std(p_est[ic])},
                                             ignore_index=True)
        f1_scores = f1_scores.append({'p': p, 'method': ic, 'mean_f1': np.mean(f1[ic]), 'std_f1': np.std(f1[ic])},
                                     ignore_index=True)

KeyboardInterrupt: 

In [16]:
f1_scores

,p,method,mean_f1,std_f1
0,2,real,0.640093,0.150922
1,2,aic,0.449751,0.165306
2,2,bic,0.449751,0.165306
3,2,hqic,0.449751,0.165306
4,4,real,0.406764,0.090916
5,4,aic,0.299807,0.077746
6,4,bic,0.299807,0.077746
7,4,hqic,0.299807,0.077746


In [17]:
plt.figure(figsize=(8,8))
plt.title('f1 scores')
plt.xlabel('p')
plt.ylabel('f1')

handles = []
labels = []
real, = plt.errorbar(x=f1_scores.loc[f1_scores['method']=='real', 'p'],
                     y=f1_scores.loc[f1_scores['method']=='real', 'mean_f1'],
                     yerr=f1_scores.loc[f1_scores['method']=='real', 'std_f1'],
                     label='real')
handles.append(real)
labels.append('real')
for ic in ics:
    handle, label = plt.errorbar(x=f1_scores.loc[f1_scores['method']==ic, 'p'],
                                 y=f1_scores.loc[f1_scores['method']==ic, 'mean_f1'],
                                 yerr=f1_scores.loc[f1_scores['method']==ic, 'std_f1'],
                                 label=ic)
    handles.append(handle)
    labels.append(label)
plt.legend(handles, labels)
plt.show()

ValueError: too many values to unpack (expected 1)

In [18]:
plt.figure(figsize=(8,8))
plt.title('p estimations')
plt.xlabel('p')
plt.ylabel('p estimation')

handles = []
labels = []
real, = plt.errorbar(x=max_ps,
                     y=max_ps,
                     yerr=0,
                     label='real')
handles.append(real)
labels.append('real')
for ic in ics:
    handle, label = plt.errorbar(x=p_estimations.loc[p_estimations['method']==ic, 'p'],
                                 y=p_estimations.loc[p_estimations['method']==ic, 'mean_p'],
                                 yerr=p_estimations.loc[p_estimations['method']==ic, 'std_p'],
                                 label=ic)
    handles.append(handle)
    labels.append(label)
plt.legend(handles, labels)
plt.show()

ValueError: too many values to unpack (expected 1)